In [1]:
import pandas as pd
import numpy as np
import torch
import pyro
import pyro.distributions as dist
import pyro.infer
import pyro.optim
from pyro.nn import PyroSample, PyroModule
from pyro.infer.autoguide import AutoDiagonalNormal, AutoLowRankMultivariateNormal, AutoMultivariateNormal
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os





In [2]:
df = pd.read_csv(r'/Users/jamesjirsa/Desktop/Data_Science/JJ_Projects/UFC/new_method/scrape_ufc_stats/final_fight_data.csv')
df

,Unnamed: 0,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,...,delta_ELO_FIGHTER,delta_AGE,delta_REACH,delta_HEIGHT,delta_SIG_STR_vs_avoid,delta_TD_vs_avoid,PC1,PC2,opponent_PC1,opponent_PC2
0,0,UFC25:UltimateJapan3,TitoOrtizvs.WanderleiSilva,W/L,UFC Light Heavyweight Title Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),John McCarthy,...,-45.805984,-1.442847,0.0,-4.0,5.852806,-0.025210,-2309.770770,111.777029,123.244200,65.610081
1,1,UFC25:UltimateJapan3,TitoOrtizvs.WanderleiSilva,W/L,UFC Light Heavyweight Title Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),John McCarthy,...,45.805984,1.442847,0.0,4.0,0.812415,-0.099214,123.244200,65.610081,-2309.770770,111.777029
2,4,UFC27:UltimateBadBoyz,JeremyHornvs.EugeneJackson,W/L,Middleweight Bout,Submission,1,4:32,3 Rnd (5-5-5),Mario Yamasaki,...,22.284177,-8.919918,NaN,5.0,-0.581752,0.029975,-846.219711,-360.531587,NaN,NaN
3,5,UFC28:HighStakes,RandyCouturevs.KevinRandleman,W/L,UFC Heavyweight Title Bout,KO/TKO,3,4:13,5 Rnd (5-5-5-5-5),John McCarthy,...,44.264592,8.134155,NaN,3.0,1.683676,0.236066,-1103.921141,-426.244419,NaN,NaN
4,6,UFC28:HighStakes,JensPulvervs.JohnLewis,W/L,Lightweight Bout,KO/TKO,1,0:15,3 Rnd (5-5-5),Mario Yamasaki,...,27.873931,NaN,NaN,-5.0,0.894140,-0.006089,-310.385372,125.252047,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10398,11693,UFCFightNight:Sandhagenvs.Nurmagomedov,ViktoriiaDudakovavs.SamHughes,L/W,Women's Strawweight Bout,Decision - Split,3,5:00,3 Rnd (5-5-5),Daniel Movahedi,...,-53.379668,6.579055,NaN,NaN,-3.026368,0.064299,4525.143067,-1018.395761,NaN,NaN
10399,11694,UFCFightNight:Sandhagenvs.Nurmagomedov,SharaMagomedovvs.MichalOleksiejczuk,W/L,Middleweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Daniel Movahedi,...,-24.699481,-0.772074,NaN,-2.0,3.331041,-0.523622,2107.863170,-1553.580021,NaN,NaN
10400,11695,UFCFightNight:Sandhagenvs.Nurmagomedov,TonyFergusonvs.MichaelChiesa,L/W,Welterweight Bout,Submission,1,3:44,3 Rnd (5-5-5),Marc Goddard,...,44.625080,-3.816564,-1.0,2.0,-3.546148,0.076678,8868.919814,-158.645274,13129.879716,-6580.190969
10401,11696,UFCFightNight:Sandhagenvs.Nurmagomedov,MarlonVeravs.DeivesonFigueiredo,L/W,Bantamweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Keith Peterson,...,-57.771347,-4.958248,2.0,3.0,0.794614,-0.137185,15885.407700,-6412.125198,9011.455859,-2906.188676


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10403 entries, 0 to 10402
Columns: 177 entries, Unnamed: 0 to opponent_PC2
dtypes: float64(159), int64(4), object(14)
memory usage: 14.0+ MB


In [4]:
max_fight_idx = df.groupby('FIGHTER')['Fight_Number'].idxmax()
max_fight_rows = df.loc[max_fight_idx]
extra = max_fight_rows[['FIGHTER', 'PC1', 'PC2']]
extra = extra.reset_index(drop=True)
extra

,FIGHTER,PC1,PC2
0,AJDobson,203.796394,-684.489701
1,AJFletcher,-186.817878,-240.184326
2,AaronPhillips,-776.704717,-475.177970
3,AaronRiley,2578.040225,-922.712749
4,AaronRosa,-912.749506,-419.983606
...,...,...,...
1539,ZhalgasZhumagulov,572.936588,-812.903884
1540,ZhangLipeng,360.553828,-117.202185
1541,ZhangTiequan,-2470.537479,181.515467
1542,ZhangWeili,5546.337955,-1268.547434


In [5]:
df = df.drop(['PC1', 'PC2'], axis = 1)

In [6]:
df['RESULT'] = np.where(df['RESULT'] == 'Win', 1,0)

In [7]:
features = df.columns[df.columns.get_loc('ELO_FIGHTER'):].tolist()
#features.remove('CUM_index')
features.remove('DOB')
df = df.dropna(subset = 'opponent_PC1')
#df = df.drop(['CUM_index'], axis = 1)

In [8]:
cum_features = [f for f in features if f.startswith('CUM_') and not (f.endswith('_PM') or f.endswith('_received'))]
pm_features = [f for f in features if f.endswith('_PM') and not f.startswith('delta_')]
delta_features = [f for f in features if f.startswith('delta_')]
received_features = [f for f in features if f.endswith('_received')]

# Fight-related features (non-CUM, non-delta, non-_received)
fight_features = ['TOTAL STR._Landed', 'TOTAL STR._Attempted', 'TOTAL STR._Attempted_avoided', 
                  'TD_Landed', 'TD_Attempted', 'TD_Attempted_avoided', 
                  'HEAD_Landed', 'HEAD_Attempted', 'HEAD_Attempted_avoided', 
                  'BODY_Landed', 'BODY_Attempted', 'BODY_Attempted_avoided', 
                  'LEG_Landed', 'LEG_Attempted', 'LEG_Attempted_avoided', 
                  'DISTANCE_Landed', 'DISTANCE_Attempted', 'DISTANCE_Attempted_avoided', 
                  'CLINCH_Landed', 'CLINCH_Attempted', 'CLINCH_Attempted_avoided',
                   'KD',
                    'SIG.STR._Landed',
                    'SIG.STR._Attempted',
                    'SIG.STR._Attempted_avoided',
                    'GROUND_Landed',
                    'GROUND_Attempted',
                    'GROUND_Attempted_avoided',
                    'CTRL_TIME_IN_SECONDS',
                    'SUB.ATT',
                    'REV.',]

# Remaining features not in other categories
remaining_features = [f for f in features if f not in cum_features + pm_features + delta_features + received_features + fight_features]


In [9]:
received_features

['CUM_SIG.STR._Landed_received',
 'CUM_TOTAL STR._Landed_received',
 'CUM_TD_Landed_received',
 'CUM_HEAD_Landed_received',
 'CUM_BODY_Landed_received',
 'CUM_LEG_Landed_received',
 'CUM_DISTANCE_Landed_received',
 'CUM_CLINCH_Landed_received',
 'CUM_GROUND_Landed_received',
 'SIG.STR._Landed_received',
 'TOTAL STR._Landed_received',
 'TD_Landed_received',
 'HEAD_Landed_received',
 'BODY_Landed_received',
 'LEG_Landed_received',
 'DISTANCE_Landed_received',
 'CLINCH_Landed_received',
 'GROUND_Landed_received']

In [10]:
len(cum_features) + len(pm_features) + len(delta_features) + len(received_features) + len(fight_features) + len(remaining_features)

158

In [11]:
len(features)

158

In [12]:
abc = df[(df['DATE'] < '2005')]
abc = abc[features]


X_train = df[(df['DATE'] >= '2005') & (df['DATE'] < '2023')]
y_train = X_train['RESULT']
print(X_train.DATE.max())
X_train = X_train[features]

X_test = df[(df['DATE'] >= '2023') & (df['DATE'] < '2024')]
y_test = X_test['RESULT']
X_test = X_test[features]

X_val = df[(df['DATE'] >= '2024')]
y_val = X_val['RESULT']
X_val = X_val[features]


2022-12-17


In [13]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)
abc_scaled = scaler.transform(abc)

In [14]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns = X_test.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns = X_val.columns)
abc_scaled  = pd.DataFrame(abc_scaled, columns = abc.columns)

In [15]:
X_train.shape

(9020, 158)

In [16]:
X_train_scaled

,ELO_FIGHTER,CUM_KD,CUM_SIG.STR._Landed,CUM_SIG.STR._Attempted,CUM_SIG.STR._Attempted_avoided,CUM_TOTAL STR._Landed,CUM_TOTAL STR._Attempted,CUM_TOTAL STR._Attempted_avoided,CUM_TD_Landed,CUM_TD_Attempted,...,delta_CUM_GROUND_Landed_PM,delta_CUM_SUB.ATT_PM,delta_ELO_FIGHTER,delta_AGE,delta_REACH,delta_HEIGHT,delta_SIG_STR_vs_avoid,delta_TD_vs_avoid,opponent_PC1,opponent_PC2
0,0.248139,0.769543,-0.504221,-0.546830,-0.565189,-0.638842,-0.631079,-0.591191,-0.685737,-0.692386,...,-0.414060,0.446222,0.144171,-0.563464,-0.913979,-1.995668,-0.159784,-0.729390,-0.901345,0.638326
1,-0.719262,-0.369616,-0.659646,-0.629953,-0.713998,-0.552265,-0.586917,-0.724418,-0.504929,-0.580706,...,-0.155445,-0.125488,-0.462714,1.549790,-1.218638,0.399134,-0.131731,0.105043,-0.591187,0.557743
2,-0.318515,-0.749336,-0.830613,-0.794502,-0.616817,-0.798468,-0.794907,-0.617837,0.308707,0.014924,...,0.155445,0.125488,0.462714,-1.549790,1.218638,-0.399134,-0.063439,1.923466,-0.554601,0.266547
3,0.123275,0.010103,-0.360453,-0.506117,-0.762589,-0.549559,-0.604012,-0.783630,-0.685737,-0.729613,...,0.414060,-0.446222,-0.144171,0.563464,0.913979,1.995668,3.597300,-1.220989,-0.638712,0.156029
4,1.303808,-0.369616,0.560441,0.231811,-0.197720,1.955767,1.045662,-0.173746,2.659211,1.392317,...,0.543198,-0.639070,0.630973,0.422334,0.000000,1.197401,0.534409,1.193470,-0.452746,-0.066829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9015,0.846508,-0.369616,-0.197256,-0.226213,-0.422453,-0.279006,-0.253563,-0.431319,-0.595333,-0.059530,...,-0.195186,-0.858231,0.873302,0.696665,0.304660,0.000000,1.297544,-0.832062,-0.619763,0.822012
9016,0.090158,-0.749336,-0.725701,-0.767360,-0.832439,-0.425104,-0.575520,-0.783630,0.308707,0.089377,...,0.195186,0.858231,-0.873302,-0.696665,-0.304660,0.000000,-0.222487,1.597109,-0.412399,-0.114677
9017,-1.059828,-0.369616,-0.593590,-0.501028,-0.486229,-0.738947,-0.609710,-0.511255,-0.595333,-0.617933,...,-0.090685,0.209981,-0.428331,0.052329,0.000000,-0.399134,-0.384122,-0.578205,-0.366346,0.513558
9018,1.824815,0.769543,3.525174,3.838325,3.889988,2.502285,3.265172,3.814184,0.670323,0.201058,...,0.028789,0.142765,0.777682,-1.342059,-0.304660,0.798267,0.226601,-0.526152,1.079822,-2.159894


In [17]:

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        """
        Arguments:
        patience: How many epochs to wait after last time validation loss improved.
        min_delta: Minimum change in the monitored quantity to qualify as an improvement.
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the autoencoder class
class Autoencoder(nn.Module):
    def __init__(self, input_size, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, encoding_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_size),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Create autoencoders for each feature group
cum_autoencoder = Autoencoder(input_size=len(cum_features), encoding_dim=3)
pm_autoencoder = Autoencoder(input_size=len(pm_features), encoding_dim=3)
delta_autoencoder = Autoencoder(input_size=len(delta_features), encoding_dim=3)
received_autoencoder = Autoencoder(input_size=len(received_features), encoding_dim=3)
fight_autoencoder = Autoencoder(input_size=len(fight_features), encoding_dim=3)
remaining_autoencoder = Autoencoder(input_size=len(remaining_features), encoding_dim=3)

# Loss function
criterion = nn.MSELoss()

# Optimizers for each autoencoder
optim_cum = optim.Adam(cum_autoencoder.parameters(), lr=1e-3)
optim_pm = optim.Adam(pm_autoencoder.parameters(), lr=1e-3)
optim_delta = optim.Adam(delta_autoencoder.parameters(), lr=1e-3)
optim_received = optim.Adam(received_autoencoder.parameters(), lr=1e-3)
optim_fight = optim.Adam(fight_autoencoder.parameters(), lr=1e-3)
optim_remaining = optim.Adam(remaining_autoencoder.parameters(), lr=1e-3)

# Training function
def train_autoencoder_with_early_stopping(autoencoder, optimizer, feature_data, val_data, epochs=2000, patience=10):
    feature_tensor = torch.tensor(feature_data, dtype=torch.float)
    val_tensor = torch.tensor(val_data, dtype=torch.float)
    
    early_stopping = EarlyStopping(patience=patience, min_delta=0.001)
    for epoch in range(epochs):
        autoencoder.train()
        encoded, decoded = autoencoder(feature_tensor)
        loss = criterion(decoded, feature_tensor)  # training loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Validation step
        autoencoder.eval()  # set to evaluation mode
        with torch.no_grad():
            val_encoded, val_decoded = autoencoder(val_tensor)
            val_loss = criterion(val_decoded, val_tensor)  # validation loss

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')
        
        # Check early stopping condition
        early_stopping(val_loss.item())
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break

# Train each autoencoder on their respective scaled feature sets from X_train_scaled
train_autoencoder_with_early_stopping(pm_autoencoder, optim_pm, X_train_scaled[pm_features].values, abc_scaled[pm_features].values, epochs=2000, patience=100)
train_autoencoder_with_early_stopping(delta_autoencoder, optim_delta, X_train_scaled[delta_features].values, abc_scaled[delta_features].values, epochs=2000, patience=100)
train_autoencoder_with_early_stopping(received_autoencoder, optim_received, X_train_scaled[received_features].values, abc_scaled[received_features].values, epochs=2000, patience=100)
train_autoencoder_with_early_stopping(fight_autoencoder, optim_fight, X_train_scaled[fight_features].values, abc_scaled[fight_features].values, epochs=2000, patience=10)
train_autoencoder_with_early_stopping(remaining_autoencoder, optim_remaining, X_train_scaled[remaining_features].values, abc_scaled[remaining_features].values, epochs=2000, patience=100)

# Encoding the scaled feature sets after training
with torch.no_grad():
    cum_encoded, _ = cum_autoencoder(torch.tensor(X_train_scaled[cum_features].values, dtype=torch.float))
    pm_encoded, _ = pm_autoencoder(torch.tensor(X_train_scaled[pm_features].values, dtype=torch.float))
    delta_encoded, _ = delta_autoencoder(torch.tensor(X_train_scaled[delta_features].values, dtype=torch.float))
    received_encoded, _ = received_autoencoder(torch.tensor(X_train_scaled[received_features].values, dtype=torch.float))
    fight_encoded, _ = fight_autoencoder(torch.tensor(X_train_scaled[fight_features].values, dtype=torch.float))
    remaining_encoded, _ = remaining_autoencoder(torch.tensor(X_train_scaled[remaining_features].values, dtype=torch.float))

# Concatenate all encoded features into a single tensor
encoded_features = torch.cat([cum_encoded, pm_encoded, delta_encoded, received_encoded, fight_encoded, remaining_encoded], dim=1)


Epoch [1/2000], Loss: 1.0121, Val Loss: 1.9554
Epoch [2/2000], Loss: 0.9975, Val Loss: 1.9372
Epoch [3/2000], Loss: 0.9848, Val Loss: 1.9195
Epoch [4/2000], Loss: 0.9734, Val Loss: 1.9011
Epoch [5/2000], Loss: 0.9626, Val Loss: 1.8814
Epoch [6/2000], Loss: 0.9518, Val Loss: 1.8601
Epoch [7/2000], Loss: 0.9406, Val Loss: 1.8367
Epoch [8/2000], Loss: 0.9287, Val Loss: 1.8105
Epoch [9/2000], Loss: 0.9156, Val Loss: 1.7811
Epoch [10/2000], Loss: 0.9012, Val Loss: 1.7486
Epoch [11/2000], Loss: 0.8855, Val Loss: 1.7130
Epoch [12/2000], Loss: 0.8683, Val Loss: 1.6747
Epoch [13/2000], Loss: 0.8496, Val Loss: 1.6342
Epoch [14/2000], Loss: 0.8296, Val Loss: 1.5917
Epoch [15/2000], Loss: 0.8083, Val Loss: 1.5477
Epoch [16/2000], Loss: 0.7859, Val Loss: 1.5029
Epoch [17/2000], Loss: 0.7628, Val Loss: 1.4579
Epoch [18/2000], Loss: 0.7392, Val Loss: 1.4137
Epoch [19/2000], Loss: 0.7156, Val Loss: 1.3711
Epoch [20/2000], Loss: 0.6924, Val Loss: 1.3313
Epoch [21/2000], Loss: 0.6701, Val Loss: 1.2954
E

In [19]:
# Concatenate encoded features after autoencoder training
with torch.no_grad():
    cum_encoded, _ = cum_autoencoder(torch.tensor(X_train_scaled[cum_features].values, dtype=torch.float))
    pm_encoded, _ = pm_autoencoder(torch.tensor(X_train_scaled[pm_features].values, dtype=torch.float))
    delta_encoded, _ = delta_autoencoder(torch.tensor(X_train_scaled[delta_features].values, dtype=torch.float))
    received_encoded, _ = received_autoencoder(torch.tensor(X_train_scaled[received_features].values, dtype=torch.float))
    fight_encoded, _ = fight_autoencoder(torch.tensor(X_train_scaled[fight_features].values, dtype=torch.float))
    remaining_encoded, _ = remaining_autoencoder(torch.tensor(X_train_scaled[remaining_features].values, dtype=torch.float))

    # Concatenate all encoded feature tensors
    encoded_train_features = torch.cat([cum_encoded, pm_encoded, delta_encoded, received_encoded, fight_encoded, remaining_encoded], dim=1)

# Do the same for the validation set
with torch.no_grad():
    cum_encoded_val, _ = cum_autoencoder(torch.tensor(X_val_scaled[cum_features].values, dtype=torch.float))
    pm_encoded_val, _ = pm_autoencoder(torch.tensor(X_val_scaled[pm_features].values, dtype=torch.float))
    delta_encoded_val, _ = delta_autoencoder(torch.tensor(X_val_scaled[delta_features].values, dtype=torch.float))
    received_encoded_val, _ = received_autoencoder(torch.tensor(X_val_scaled[received_features].values, dtype=torch.float))
    fight_encoded_val, _ = fight_autoencoder(torch.tensor(X_val_scaled[fight_features].values, dtype=torch.float))
    remaining_encoded_val, _ = remaining_autoencoder(torch.tensor(X_val_scaled[remaining_features].values, dtype=torch.float))

    # Concatenate all encoded feature tensors for validation
    encoded_val_features = torch.cat([cum_encoded_val, pm_encoded_val, delta_encoded_val, received_encoded_val, fight_encoded_val, remaining_encoded_val], dim=1)

# Do the same for the test set
with torch.no_grad():
    cum_encoded_test, _ = cum_autoencoder(torch.tensor(X_test_scaled[cum_features].values, dtype=torch.float))
    pm_encoded_test, _ = pm_autoencoder(torch.tensor(X_test_scaled[pm_features].values, dtype=torch.float))
    delta_encoded_test, _ = delta_autoencoder(torch.tensor(X_test_scaled[delta_features].values, dtype=torch.float))
    received_encoded_test, _ = received_autoencoder(torch.tensor(X_test_scaled[received_features].values, dtype=torch.float))
    fight_encoded_test, _ = fight_autoencoder(torch.tensor(X_test_scaled[fight_features].values, dtype=torch.float))
    remaining_encoded_test, _ = remaining_autoencoder(torch.tensor(X_test_scaled[remaining_features].values, dtype=torch.float))

    # Concatenate all encoded feature tensors for test
    encoded_test_features = torch.cat([cum_encoded_test, pm_encoded_test, delta_encoded_test, received_encoded_test, fight_encoded_test, remaining_encoded_test], dim=1)

with torch.no_grad():
    # Autoencode the features in abc_scaled for each feature group
    cum_encoded_abc, _ = cum_autoencoder(torch.tensor(abc_scaled[cum_features].values, dtype=torch.float))
    pm_encoded_abc, _ = pm_autoencoder(torch.tensor(abc_scaled[pm_features].values, dtype=torch.float))
    delta_encoded_abc, _ = delta_autoencoder(torch.tensor(abc_scaled[delta_features].values, dtype=torch.float))
    received_encoded_abc, _ = received_autoencoder(torch.tensor(abc_scaled[received_features].values, dtype=torch.float))
    fight_encoded_abc, _ = fight_autoencoder(torch.tensor(abc_scaled[fight_features].values, dtype=torch.float))
    remaining_encoded_abc, _ = remaining_autoencoder(torch.tensor(abc_scaled[remaining_features].values, dtype=torch.float))

    # Concatenate all encoded feature tensors for abc_scaled
    encoded_abc_features = torch.cat([cum_encoded_abc, pm_encoded_abc, delta_encoded_abc, received_encoded_abc, fight_encoded_abc, remaining_encoded_abc], dim=1)



In [20]:
encoded_train_features

tensor([[ 0.1397,  0.0487,  0.1281,  ...,  1.1624,  0.6920, -1.2212],
        [ 0.1587,  0.0219,  0.0122,  ..., -0.4199,  0.6297,  0.1459],
        [ 0.1335, -0.0168,  0.0246,  ..., -0.3717,  0.3906,  0.1171],
        ...,
        [ 0.0582,  0.0116,  0.0631,  ..., -0.7518, -0.7134,  0.2909],
        [-0.0047,  0.9855,  0.3002,  ...,  4.0577, -1.0446,  1.0318],
        [-0.0231, -0.1708,  0.1131,  ...,  1.1970, -3.5174, -2.6160]])

In [21]:
import torch
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import torch.nn.functional as F
import torch.nn as nn

pyro.clear_param_store()

class SimpleBayesianNN(PyroModule):
    def __init__(self, input_size, hidden_size, output_size, means, stds):
        super().__init__()

        self.hidden1 = PyroModule[torch.nn.Linear](input_size, hidden_size)
        self.hidden1.weight = PyroSample(dist.Normal(means, stds).expand([hidden_size, input_size]).to_event(2))
        self.hidden1.bias = PyroSample(dist.Normal(means.mean(), stds.mean()).expand([hidden_size]).to_event(1))

        # Residual layer to match dimensions between x and hidden1
        self.residual_layer = PyroModule[torch.nn.Linear](input_size, hidden_size)

        self.output = PyroModule[torch.nn.Linear](hidden_size, output_size)
        self.output.weight = PyroSample(dist.Normal(0., .1).expand([output_size, hidden_size]).to_event(2))
        self.output.bias = PyroSample(dist.Normal(0., .1).expand([output_size]).to_event(1))

    def forward(self, x):
        hidden1 = F.leaky_relu(self.hidden1(x))
        hidden1 = F.dropout(hidden1, p=0.5, training=self.training)
        
        residual = self.residual_layer(x)  # Match dimensions for residual connection
        output = self.output(hidden1 + residual)
        return output
    

def model(x_data, y_data=None):
    logits = bnn(x_data).squeeze(-1)
    probabilities = torch.sigmoid(logits)  
    with pyro.plate("data", x_data.shape[0]):
        pyro.sample("obs", dist.Bernoulli(probs=probabilities), obs=y_data)
    return logits


means = encoded_abc_features.mean(dim=0)
stds = encoded_abc_features.std(dim=0)

guide = pyro.infer.autoguide.AutoDiagonalNormal(model)  

bnn = SimpleBayesianNN(input_size=encoded_train_features.shape[1], hidden_size=200, output_size=1, means=means, stds=stds)


# Use encoded training, validation, and test data in the BNN
x_tensor = encoded_train_features  # Encoded training features
y_tensor = torch.tensor(y_train.values, dtype=torch.float)

x_tensor_val = encoded_val_features  # Encoded validation features
y_tensor_val = torch.tensor(y_val.values, dtype=torch.float)

x_tensor_test = encoded_test_features  # Encoded test features
y_tensor_test = torch.tensor(y_test.values, dtype=torch.float)


In [22]:
x_tensor

tensor([[ 0.1397,  0.0487,  0.1281,  ...,  1.1624,  0.6920, -1.2212],
        [ 0.1587,  0.0219,  0.0122,  ..., -0.4199,  0.6297,  0.1459],
        [ 0.1335, -0.0168,  0.0246,  ..., -0.3717,  0.3906,  0.1171],
        ...,
        [ 0.0582,  0.0116,  0.0631,  ..., -0.7518, -0.7134,  0.2909],
        [-0.0047,  0.9855,  0.3002,  ...,  4.0577, -1.0446,  1.0318],
        [-0.0231, -0.1708,  0.1131,  ...,  1.1970, -3.5174, -2.6160]])

In [23]:
class EarlyStopper:
    def __init__(self, patience=500, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False
    

def save_checkpoint(model, pyro_optimizer, scheduler, epoch, loss, path="checkpoint.pth"):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': pyro_optimizer.get_state(),  
        'scheduler_state_dict': scheduler.scheduler.state_dict(),  
        'loss': loss
    }, path)
    print(f"Checkpoint saved at epoch {epoch}.")

def load_checkpoint(model, pyro_optimizer, scheduler, path="checkpoint.pth"):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    pyro_optimizer.set_state(checkpoint['optimizer_state_dict'])  
    scheduler.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])  
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"Checkpoint loaded. Resuming from epoch {epoch} with loss {loss:.4f}")
    return epoch, loss

def load_model_only(model, path="checkpoint.pth"):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])


def save_best_model(model, val_loss, best_val_loss, model_save_path="best_model.pth"):
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), model_save_path)
        print(f"Best model saved with val_loss: {val_loss:.4f}")
    return best_val_loss

def load_best_model(model, model_save_path="best_model.pth"):
    model.load_state_dict(torch.load(model_save_path))
    model.eval()  # Set to evaluation mode
    print("Best model loaded.")


early_stopper = EarlyStopper(patience=2000, min_delta=0.01)

In [24]:
from pyro.optim import PyroOptim
from pyro.optim.lr_scheduler import PyroLRScheduler
import torch.optim as optim

optimizer = PyroOptim(optim.Adam, {"lr": 1e-3, "weight_decay": 1e-3})
scheduler = pyro.optim.ReduceLROnPlateau({
    'optimizer': optimizer,
    'optim_args': {'lr': 0.01},  # Learning rate arguments
    'factor': 1,               # Learning rate reduction factor
    'patience': 50,              # Number of epochs with no improvement before reducing LR
    'verbose': True,             # Enable logging of LR updates
    'mode': 'min',               # Reduce LR when a monitored metric has stopped decreasing
    'gamma': 0.1                 # The same gamma factor
})


svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

def focal_loss(logits, targets, alpha=0.25, gamma=2):
    BCE_loss = F.binary_cross_entropy_with_logits(logits, targets, reduction='none')
    pt = torch.exp(-BCE_loss)
    F_loss = alpha * (1-pt)**gamma * BCE_loss
    return F_loss.mean()

def compute_validation_loss(x_test, y_test):
    with torch.no_grad():
        logits = bnn(x_test).squeeze(-1)
        val_loss = focal_loss(logits, y_test)
    return val_loss.item()

def compute_validation_loss_no_foc(x_test, y_test):
    with torch.no_grad():
        logits = bnn(x_test).squeeze(-1)
        val_loss = F.binary_cross_entropy_with_logits(logits, y_test)
    return val_loss.item()


best_val_loss = float('inf')
num_epochs = 2000
checkpoint_path = 'best_model_AE.pth'

for epoch in range(num_epochs):
    train_loss = svi.step(x_tensor, y_tensor)
    val_loss = compute_validation_loss_no_foc(x_tensor_test, y_tensor_test)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(bnn.state_dict(), "best_model_AE.pth")  
        print(f"Model saved at epoch {epoch} with validation loss {val_loss}")


    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}, Best is = {best_val_loss:.4f}")

bnn.load_state_dict(torch.load("best_model_AE.pth"))
bnn.eval()


predictive = pyro.infer.Predictive(model, guide=guide, num_samples=5000, return_sites=["_RETURN", 'obs'])
predictions = predictive(x_tensor, None)
logits_samples = predictions["_RETURN"]
probability_samples = torch.sigmoid(logits_samples)
mean_probabilities = torch.mean(probability_samples, dim=0)
predicted_labels = (mean_probabilities > 0.5).float()
accuracy = (predicted_labels == y_tensor).float().mean()
accuracy


Model saved at epoch 0 with validation loss 7.925364017486572
Epoch 0: Train Loss = 33443.0229, Val Loss = 7.9254, Best is = 7.9254
Model saved at epoch 2 with validation loss 7.426735877990723
Model saved at epoch 3 with validation loss 6.118226528167725
Model saved at epoch 7 with validation loss 5.821559429168701
Model saved at epoch 9 with validation loss 5.725302696228027
Model saved at epoch 10 with validation loss 5.664477348327637
Model saved at epoch 32 with validation loss 5.159912109375
Epoch 100: Train Loss = 35233.7414, Val Loss = 5.5330, Best is = 5.1599
Epoch 200: Train Loss = 41798.2016, Val Loss = 7.3881, Best is = 5.1599
Model saved at epoch 237 with validation loss 4.951593399047852
Model saved at epoch 266 with validation loss 4.870832443237305
Epoch 300: Train Loss = 29699.3356, Val Loss = 10.0882, Best is = 4.8708
Model saved at epoch 341 with validation loss 4.727814674377441
Epoch 400: Train Loss = 29781.3361, Val Loss = 7.1684, Best is = 4.7278
Epoch 500: Train

tensor(0.5501)

In [25]:
lower_bound = torch.quantile(probability_samples, 0.025, dim=0)  
upper_bound = torch.quantile(probability_samples, 0.975, dim=0)
std = torch.std(probability_samples, dim=0)
skew = torch.mean(((probability_samples - mean_probabilities) ** 3), dim=0) / (std ** 3)
kurtosis = torch.mean(((probability_samples - mean_probabilities) ** 4), dim=0) / (std ** 4)


In [26]:
data = np.column_stack((mean_probabilities.numpy(), lower_bound.numpy(), upper_bound.numpy(), std.numpy(), skew.numpy(), kurtosis.numpy()))
datafr = pd.DataFrame(data, columns = ['proba', 'lower', 'upper', 'std', 'skew', 'kurtosis'])
datafr

,proba,lower,upper,std,skew,kurtosis
0,0.297021,5.337259e-02,0.704958,0.173565,0.746071,3.021590
1,0.728791,3.467245e-04,1.000000,0.381522,-1.026297,2.307832
2,0.752906,6.821339e-03,0.999990,0.340500,-1.160038,2.773231
3,0.760682,2.911244e-11,1.000000,0.407884,-1.221747,2.573428
4,0.788893,3.637199e-08,1.000000,0.383072,-1.411269,3.120392
...,...,...,...,...,...,...
9015,0.503126,3.525460e-01,0.654097,0.077337,0.028352,2.876169
9016,0.763365,2.576610e-12,1.000000,0.408574,-1.238908,2.607782
9017,0.490445,3.691326e-01,0.617458,0.064624,0.013716,2.781718
9018,0.459864,2.023696e-01,0.736926,0.140523,0.113887,2.501379


In [27]:
predictions = predictive(x_tensor_val, None)

In [28]:
logits_samples = predictions["_RETURN"]
probability_samples = torch.sigmoid(logits_samples)
mean_probabilities = torch.mean(probability_samples, dim=0)
predicted_labels = (mean_probabilities > 0.5).float()

accuracy = (predicted_labels == y_tensor_val).float().mean()
accuracy

tensor(0.5473)

In [29]:
lower_bound = torch.quantile(probability_samples, 0.025, dim=0)  
upper_bound = torch.quantile(probability_samples, 0.975, dim=0)
std = torch.std(probability_samples, dim=0)
skew = torch.mean(((probability_samples - mean_probabilities) ** 3), dim=0) / (std ** 3)
kurtosis = torch.mean(((probability_samples - mean_probabilities) ** 4), dim=0) / (std ** 4)
median = torch.median(probability_samples, dim=0)


datafr = {
    'proba': mean_probabilities.numpy(),
    'median': median.values.numpy(),
    'lower': lower_bound.numpy(),
    'upper': upper_bound.numpy(),
    'std': std.numpy(),
    'skew': skew.numpy(),
    'kurtosis': kurtosis.numpy(),
    'trace': [probability_samples[i] for i in range(mean_probabilities.shape[0])] 
}

datafr = pd.DataFrame(datafr)

datafr = datafr.reset_index(drop =  True)

extra_val = df[(df['DATE'] >= '2024')]
extra_val = extra_val.reset_index(drop = True)

xtra_feats =['EVENT',
 'BOUT',
 'OUTCOME',
 'WEIGHTCLASS',
 'METHOD',
 'ROUND',
 'TIME',
 'TIME FORMAT',
 'REFEREE',
 'DETAILS',
 'URL',
 'FIGHTER',
 'RESULT',
 'TIME_IN_SECONDS',
 'DATE',
 'ELO_FIGHTER',]

extra_val = pd.concat([extra_val[xtra_feats], datafr], axis = 1)
extra_val.to_csv(r'ProbaPlus_AE.csv')
extra_val['Estimate'] = np.where(extra_val['proba'] > .5,1,0)

In [30]:
extra_val

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,...,ELO_FIGHTER,proba,median,lower,upper,std,skew,kurtosis,trace,Estimate
0,UFCFightNight:Ankalaevvs.Walker2,MagomedAnkalaevvs.JohnnyWalker,W/L,Light Heavyweight Bout,KO/TKO,2,2:42,5 Rnd (5-5-5-5-5),Marc Goddard,Punch to Head At Distance,...,1726.390024,0.669927,0.759190,0.064609,0.993731,0.288368,-0.690849,2.210196,"[tensor(0.5757), tensor(0.2977), tensor(0.4158...",1
1,UFCFightNight:Ankalaevvs.Walker2,JimMillervs.GabrielBenitez,W/L,Lightweight Bout,Submission,3,3:25,3 Rnd (5-5-5),Dan Miragliotta,Neck Crank From Back Control,...,1509.527752,0.487733,0.483612,0.130990,0.867594,0.202510,0.071405,2.165578,"[tensor(0.5206), tensor(0.3033), tensor(0.5628...",0
2,UFCFightNight:Ankalaevvs.Walker2,JimMillervs.GabrielBenitez,W/L,Lightweight Bout,Submission,3,3:25,3 Rnd (5-5-5),Dan Miragliotta,Neck Crank From Back Control,...,1606.672231,0.313769,0.274243,0.038304,0.770339,0.201599,0.708433,2.773551,"[tensor(0.9892), tensor(0.8510), tensor(0.2175...",0
3,UFCFightNight:Ankalaevvs.Walker2,MagomedAnkalaevvs.JohnnyWalker,W/L,Light Heavyweight Bout,KO/TKO,2,2:42,5 Rnd (5-5-5-5-5),Marc Goddard,Punch to Head At Distance,...,1622.106542,0.575931,0.582008,0.323868,0.799449,0.124302,-0.196757,2.651780,"[tensor(0.8763), tensor(0.5415), tensor(0.1044...",1
4,UFCFightNight:Ankalaevvs.Walker2,MatthewSemelsbergervs.PrestonParsons,L/W,Welterweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Dan Miragliotta,Mike Bell 27 - 30.Junichiro Kamijo 27 - 30.Ron...,...,1482.314337,0.811297,0.986142,0.014689,0.999997,0.311524,-1.578840,3.980455,"[tensor(0.8357), tensor(0.6041), tensor(0.2870...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,UFCFightNight:Sandhagenvs.Nurmagomedov,MackenzieDernvs.LoopyGodinez,W/L,Women's Strawweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Greg Kleynjans,Hadi Mohamed Ali 28 - 29.Howard Hughes 28 - 29...,...,1555.735723,0.372387,0.355724,0.121861,0.701482,0.153146,0.409076,2.603126,"[tensor(0.7148), tensor(0.4332), tensor(0.6254...",0
292,UFCFightNight:Sandhagenvs.Nurmagomedov,TonyFergusonvs.MichaelChiesa,L/W,Welterweight Bout,Submission,1,3:44,3 Rnd (5-5-5),Marc Goddard,Rear Naked Choke,...,1578.089046,0.330777,0.315402,0.125038,0.619703,0.128477,0.550786,2.947959,"[tensor(0.7867), tensor(0.6063), tensor(0.4453...",0
293,UFCFightNight:Sandhagenvs.Nurmagomedov,TonyFergusonvs.MichaelChiesa,L/W,Welterweight Bout,Submission,1,3:44,3 Rnd (5-5-5),Marc Goddard,Rear Naked Choke,...,1622.714126,0.647541,0.739231,0.039999,0.995035,0.307114,-0.618858,2.037477,"[tensor(0.5761), tensor(0.9245), tensor(0.2913...",1
294,UFCFightNight:Sandhagenvs.Nurmagomedov,MarlonVeravs.DeivesonFigueiredo,L/W,Bantamweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Keith Peterson,Ben Cartlidge 28 - 29.Hadi Mohamed Ali 28 - 29...,...,1684.684440,0.476529,0.467581,0.115009,0.867519,0.203825,0.113106,2.238836,"[tensor(0.2567), tensor(0.6581), tensor(0.4311...",0


In [31]:
import pandas as pd
import numpy as np
import torch


from scipy.stats import bernoulli, norm

def calculate_uncertainty(p_win, num_simulations):
    """
    Calculate uncertainty statistics for a Bernoulli outcome.
    
    Parameters:
        p_win: Probability of the fighter winning.
        num_simulations: Number of simulations run.
    
    Returns:
        variance, std_dev, and Wilson score confidence intervals (lower, upper).
    """
    variance = p_win * (1 - p_win)
    std_dev = np.sqrt(variance)
    
    # 95% confidence interval using the Wilson score interval
    z = norm.ppf(0.975)  # 1.96 for 95% CI
    denominator = 1 + (z**2 / num_simulations)
    center_adjusted_probability = p_win + (z**2 / (2 * num_simulations))
    adjusted_std_dev = np.sqrt((p_win * (1 - p_win) + (z**2 / (4 * num_simulations))) / num_simulations)
    
    lower_bound = (center_adjusted_probability - z * adjusted_std_dev) / denominator
    upper_bound = (center_adjusted_probability + z * adjusted_std_dev) / denominator
    
    return variance, std_dev, lower_bound, upper_bound

def simulate_fights_by_bout_with_uncertainty(df, bout_col, trace_col, fighter_col, num_simulations=10000):
    """
    Simulate future fights for each bout by comparing tensor traces of both fighters,
    and use Bernoulli distribution to model the fight outcome, along with uncertainty statistics.
    
    Parameters:
        df: DataFrame containing fight data.
        bout_col: Column name for BOUT (to group by).
        trace_col: Column name for fighter's trace (tensor).
        fighter_col: Column name for the fighter's name.
        num_simulations: Number of simulations to run.
    
    Returns:
        A DataFrame containing the win probability, Bernoulli outcome, and uncertainty statistics.
    """
    bout_groups = df.groupby(bout_col)
    
    probabilities = []

    for bout, group in bout_groups:
        if len(group) == 2:  # Ensure there are two fighters in the bout
            fighter_A = group.iloc[0]  # First fighter
            fighter_B = group.iloc[1]  # Second fighter
            
            # Access the trace as a tensor and convert to NumPy
            trace_A = fighter_A[trace_col].numpy()
            trace_B = fighter_B[trace_col].numpy()
            
            wins_A, wins_B = 0, 0
            
            # Monte Carlo simulation to compute win probabilities
            for _ in range(num_simulations):
                sample_A = np.random.choice(trace_A)  # Random sample from Fighter A's trace
                sample_B = np.random.choice(trace_B)  # Random sample from Fighter B's trace
                
                if sample_A > sample_B:
                    wins_A += 1
                else:
                    wins_B += 1
            
            prob_A_win = wins_A / num_simulations
            prob_B_win = wins_B / num_simulations
            
            # Simulate fight outcome using Bernoulli distribution
            fight_outcome_A = bernoulli.rvs(prob_A_win)  # 1 if Fighter A wins, 0 otherwise
            fight_outcome_B = 1 - fight_outcome_A  # 1 if Fighter B wins, 0 otherwise
            
            # Calculate uncertainty statistics for Fighter A and Fighter B
            variance_A, std_dev_A, lower_A, upper_A = calculate_uncertainty(prob_A_win, num_simulations)
            variance_B, std_dev_B, lower_B, upper_B = calculate_uncertainty(prob_B_win, num_simulations)
            
            # Assign probability, Bernoulli outcome, and uncertainty stats for each fighter in the bout
            for index, fighter in group.iterrows():
                if fighter[fighter_col] == fighter_A[fighter_col]:
                    probabilities.append({
                        'index': index,
                        'win_probability': prob_A_win,
                        'bernoulli_outcome': fight_outcome_A,  # 1 if Fighter A wins
                        'variance': variance_A,
                        'std_dev': std_dev_A,
                        'lower_CI': lower_A,
                        'upper_CI': upper_A
                    })
                elif fighter[fighter_col] == fighter_B[fighter_col]:
                    probabilities.append({
                        'index': index,
                        'win_probability': prob_B_win,
                        'bernoulli_outcome': fight_outcome_B,  # 1 if Fighter B wins
                        'variance': variance_B,
                        'std_dev': std_dev_B,
                        'lower_CI': lower_B,
                        'upper_CI': upper_B
                    })
    
    # Convert probabilities into a DataFrame
    prob_df = pd.DataFrame(probabilities).set_index('index')
    
    return prob_df


simulation_results = simulate_fights_by_bout_with_uncertainty(extra_val, 'BOUT', 'trace', 'FIGHTER')
df_merged = extra_val.join(simulation_results)

In [32]:
from copulas.multivariate import GaussianMultivariate
from copulas.univariate import GaussianKDE
from copulas.multivariate import GaussianMultivariate
from copulas.univariate import GaussianUnivariate

def simulate_fights_with_copulas(df, bout_col, trace_col, fighter_col, num_simulations=10000):
    """
    Simulate future fights using copulas by modeling dependency between fighter performances.
    
    Parameters:
        df: DataFrame containing fight data.
        bout_col: Column name for BOUT (to group by).
        trace_col: Column name for fighter's trace (tensor).
        fighter_col: Column name for the fighter's name.
        num_simulations: Number of simulations to run.
    
    Returns:
        A DataFrame containing the win probability for the fighter in the row.
    """
    bout_groups = df.groupby(bout_col)
    
    probabilities = []
    
    for bout, group in bout_groups:
        if len(group) == 2:  # Ensure there are two fighters in the bout
            fighter_A = group.iloc[0]  # First fighter
            fighter_B = group.iloc[1]  # Second fighter
            
            # Extract traces and convert to NumPy
            trace_A = fighter_A[trace_col].numpy()
            trace_B = fighter_B[trace_col].numpy()
            
            # Use simple Gaussian marginals instead of KDE
            marginal_A = GaussianUnivariate()
            marginal_B = GaussianUnivariate()
            
            try:
                marginal_A.fit(trace_A)
                marginal_B.fit(trace_B)
            except Exception as e:
                print(f"Error fitting marginals for bout {bout}: {e}")
                continue  # Skip this bout if there's a fitting issue
            
            # Combine the traces into a joint dataset
            joint_data = np.column_stack([trace_A, trace_B])
            
            # Fit a Gaussian Copula to the joint distribution
            copula = GaussianMultivariate()
            try:
                copula.fit(joint_data)
            except Exception as e:
                print(f"Error fitting copula for bout {bout}: {e}")
                continue  # Skip this bout if there's a fitting issue
            
            # Simulate joint outcomes from the copula
            simulated_outcomes = copula.sample(num_simulations)
            simulated_outcomes = np.array(simulated_outcomes)  # Ensure it's a NumPy array
            
            if simulated_outcomes.shape[1] != 2:
                print(f"Unexpected shape for simulated outcomes: {simulated_outcomes.shape}")
                continue  # Skip if the simulation results aren't correct
            
            # Extract the simulated marginals for Fighter A and Fighter B
            simulated_A = simulated_outcomes[:, 0]
            simulated_B = simulated_outcomes[:, 1]
            
            # Calculate probabilities of Fighter A winning over Fighter B
            wins_A = np.sum(simulated_A > simulated_B)
            prob_A_win = wins_A / num_simulations
            prob_B_win = 1 - prob_A_win
            
            # Assign probability for the fighter in the current row
            for index, fighter in group.iterrows():
                if fighter[fighter_col] == fighter_A[fighter_col]:
                    probabilities.append({
                        'index': index,
                        'copula_win_probability': prob_A_win
                    })
                elif fighter[fighter_col] == fighter_B[fighter_col]:
                    probabilities.append({
                        'index': index,
                        'copula_win_probability': prob_B_win
                    })
    
    # Convert probabilities into a DataFrame
    prob_df = pd.DataFrame(probabilities).set_index('index')
    
    return prob_df

probabilities_df = simulate_fights_with_copulas(df_merged, 'BOUT', 'trace', 'FIGHTER')

# Merge the probabilities back to the original DataFrame based on the index
df_merged = df_merged.join(probabilities_df)


/Users/jamesjirsa/miniforge3/envs/mma/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:6338: RuntimeWarning: divide by zero encountered in divide
  return c**2 / (c**2 - n**2)
/Users/jamesjirsa/miniforge3/envs/mma/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:6338: RuntimeWarning: divide by zero encountered in divide
  return c**2 / (c**2 - n**2)
/Users/jamesjirsa/miniforge3/envs/mma/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:6338: RuntimeWarning: divide by zero encountered in divide
  return c**2 / (c**2 - n**2)
/Users/jamesjirsa/miniforge3/envs/mma/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:718: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/Users/jamesjirsa/miniforge3/envs/mma/lib/python3.11/site-packages/scipy/stats/_continuous_distns.py:6338: RuntimeWarning: divide by zero encountered in divide
  return c**2 / (c**2 - n**2)
/Users/jamesjirsa/min

In [33]:
df_merged

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,...,kurtosis,trace,Estimate,win_probability,bernoulli_outcome,variance,std_dev,lower_CI,upper_CI,copula_win_probability
0,UFCFightNight:Ankalaevvs.Walker2,MagomedAnkalaevvs.JohnnyWalker,W/L,Light Heavyweight Bout,KO/TKO,2,2:42,5 Rnd (5-5-5-5-5),Marc Goddard,Punch to Head At Distance,...,2.210196,"[tensor(0.5757), tensor(0.2977), tensor(0.4158...",1,0.4867,0,0.249823,0.499823,0.476911,0.496500,0.4823
1,UFCFightNight:Ankalaevvs.Walker2,JimMillervs.GabrielBenitez,W/L,Lightweight Bout,Submission,3,3:25,3 Rnd (5-5-5),Dan Miragliotta,Neck Crank From Back Control,...,2.165578,"[tensor(0.5206), tensor(0.3033), tensor(0.5628...",0,0.4128,1,0.242396,0.492337,0.403186,0.422481,0.4040
2,UFCFightNight:Ankalaevvs.Walker2,JimMillervs.GabrielBenitez,W/L,Lightweight Bout,Submission,3,3:25,3 Rnd (5-5-5),Dan Miragliotta,Neck Crank From Back Control,...,2.773551,"[tensor(0.9892), tensor(0.8510), tensor(0.2175...",0,0.5872,0,0.242396,0.492337,0.577519,0.596814,0.5960
3,UFCFightNight:Ankalaevvs.Walker2,MagomedAnkalaevvs.JohnnyWalker,W/L,Light Heavyweight Bout,KO/TKO,2,2:42,5 Rnd (5-5-5-5-5),Marc Goddard,Punch to Head At Distance,...,2.651780,"[tensor(0.8763), tensor(0.5415), tensor(0.1044...",1,0.5133,1,0.249823,0.499823,0.503500,0.523089,0.5177
4,UFCFightNight:Ankalaevvs.Walker2,MatthewSemelsbergervs.PrestonParsons,L/W,Welterweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Dan Miragliotta,Mike Bell 27 - 30.Junichiro Kamijo 27 - 30.Ron...,...,3.980455,"[tensor(0.8357), tensor(0.6041), tensor(0.2870...",1,0.5127,0,0.249839,0.499839,0.502900,0.522490,0.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,UFCFightNight:Sandhagenvs.Nurmagomedov,MackenzieDernvs.LoopyGodinez,W/L,Women's Strawweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Greg Kleynjans,Hadi Mohamed Ali 28 - 29.Howard Hughes 28 - 29...,...,2.603126,"[tensor(0.7148), tensor(0.4332), tensor(0.6254...",0,0.6089,1,0.238141,0.487997,0.599295,0.618421,0.6012
292,UFCFightNight:Sandhagenvs.Nurmagomedov,TonyFergusonvs.MichaelChiesa,L/W,Welterweight Bout,Submission,1,3:44,3 Rnd (5-5-5),Marc Goddard,Rear Naked Choke,...,2.947959,"[tensor(0.7867), tensor(0.6063), tensor(0.4453...",0,0.4513,0,0.247628,0.497623,0.441567,0.461070,0.4470
293,UFCFightNight:Sandhagenvs.Nurmagomedov,TonyFergusonvs.MichaelChiesa,L/W,Welterweight Bout,Submission,1,3:44,3 Rnd (5-5-5),Marc Goddard,Rear Naked Choke,...,2.037477,"[tensor(0.5761), tensor(0.9245), tensor(0.2913...",1,0.5487,1,0.247628,0.497623,0.538930,0.558433,0.5530
294,UFCFightNight:Sandhagenvs.Nurmagomedov,MarlonVeravs.DeivesonFigueiredo,L/W,Bantamweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Keith Peterson,Ben Cartlidge 28 - 29.Hadi Mohamed Ali 28 - 29...,...,2.238836,"[tensor(0.2567), tensor(0.6581), tensor(0.4311...",0,0.3623,1,0.231039,0.480665,0.352934,0.371772,0.3764


In [34]:
df_merged.to_csv('MergedWJointProba_AE.csv')

In [35]:
print(extra_val)

                                      EVENT  \
0          UFCFightNight:Ankalaevvs.Walker2   
1          UFCFightNight:Ankalaevvs.Walker2   
2          UFCFightNight:Ankalaevvs.Walker2   
3          UFCFightNight:Ankalaevvs.Walker2   
4          UFCFightNight:Ankalaevvs.Walker2   
..                                      ...   
291  UFCFightNight:Sandhagenvs.Nurmagomedov   
292  UFCFightNight:Sandhagenvs.Nurmagomedov   
293  UFCFightNight:Sandhagenvs.Nurmagomedov   
294  UFCFightNight:Sandhagenvs.Nurmagomedov   
295  UFCFightNight:Sandhagenvs.Nurmagomedov   

                                     BOUT OUTCOME               WEIGHTCLASS  \
0          MagomedAnkalaevvs.JohnnyWalker     W/L    Light Heavyweight Bout   
1              JimMillervs.GabrielBenitez     W/L          Lightweight Bout   
2              JimMillervs.GabrielBenitez     W/L          Lightweight Bout   
3          MagomedAnkalaevvs.JohnnyWalker     W/L    Light Heavyweight Bout   
4    MatthewSemelsbergervs.PrestonParson

In [36]:
mean_probabilities.numpy()

array([0.6699274 , 0.48773324, 0.31376907, 0.57593113, 0.8112971 ,
       0.5373616 , 0.57997674, 0.43945357, 0.43517256, 0.55845594,
       0.6136654 , 0.49048722, 0.42557114, 0.4546615 , 0.33998466,
       0.49982616, 0.7368689 , 0.68928957, 0.41386324, 0.47904328,
       0.6117593 , 0.5453888 , 0.35135522, 0.41235963, 0.53032887,
       0.5524815 , 0.85125536, 0.505069  , 0.38537726, 0.6046647 ,
       0.55143183, 0.38505596, 0.58296853, 0.5525833 , 0.44703373,
       0.32672492, 0.4176081 , 0.6231255 , 0.6089352 , 0.41760707,
       0.38029486, 0.64103466, 0.43141142, 0.5233549 , 0.49390668,
       0.8023379 , 0.5775018 , 0.6294845 , 0.6002789 , 0.4033495 ,
       0.64799815, 0.45472682, 0.85415334, 0.5074542 , 0.39174864,
       0.6690643 , 0.8073959 , 0.7351551 , 0.49931964, 0.3826582 ,
       0.44524702, 0.48661685, 0.5490535 , 0.36660203, 0.32905108,
       0.5336979 , 0.49845272, 0.57608974, 0.44237882, 0.78635603,
       0.39617494, 0.31596476, 0.4323557 , 0.7384062 , 0.70688

In [37]:
print(classification_report(y_tensor_val.int(), predicted_labels))

              precision    recall  f1-score   support

           0       0.55      0.53      0.54       148
           1       0.55      0.56      0.55       148

    accuracy                           0.55       296
   macro avg       0.55      0.55      0.55       296
weighted avg       0.55      0.55      0.55       296



In [38]:
class_counts = torch.bincount(torch.tensor(y_train.values, dtype=torch.long))
class_counts

tensor([4510, 4510])

In [39]:
import statsmodels.api as sm
import pandas as pd


feats = ['delta_ELO_FIGHTER', 'delta_AGE', 'delta_REACH']

X = X_train[feats]
X = sm.add_constant(X)

y = y_train

logit_model = sm.Logit(y, X)
result = logit_model.fit()

print(result.summary())

X_test['predicted_prob'] = result.predict(sm.add_constant(X_test[feats]))
X_test['predicted_class'] = (X_test['predicted_prob'] > 0.5).astype(int)

accuracy = accuracy_score(y_test, X_test['predicted_class'])
print(f"Accuracy: {accuracy:.4f}")

Optimization terminated successfully.
         Current function value: 0.667163
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                 RESULT   No. Observations:                 9020
Model:                          Logit   Df Residuals:                     9016
Method:                           MLE   Df Model:                            3
Date:                Thu, 05 Sep 2024   Pseudo R-squ.:                 0.03749
Time:                        23:05:31   Log-Likelihood:                -6017.8
converged:                       True   LL-Null:                       -6252.2
Covariance Type:            nonrobust   LLR p-value:                2.812e-101
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const             -1.147e-17      0.022  -5.31e-16      1.000      -0.042       0.042
delta_EL

In [40]:
print(classification_report(X_test['predicted_class'], y_test))

              precision    recall  f1-score   support

           0       0.57      0.57      0.57       272
           1       0.57      0.57      0.57       272

    accuracy                           0.57       544
   macro avg       0.57      0.57      0.57       544
weighted avg       0.57      0.57      0.57       544

